In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm      
from scipy import stats           
from sklearn import linear_model  
from sklearn import model_selection
from libpysal import weights
from libpysal import cg as geometry

from shapely.geometry import Polygon
from shapely.wkt import loads  

import matplotlib.pyplot as plt
import palettable.colorbrewer.sequential as pcs  
import seaborn as sns

import os
import urllib
import zipfile
import re
import pysal as ps
from pysal.model.spreg import OLS
from pysal.model.spreg import ML_Error
from pysal.model.spreg import ML_Lag

import libpysal as lps
from libpysal.weights import Queen, Rook
import pysal.viz as viz
%matplotlib inline
import warnings
warnings.simplefilter('ignore')


In [ ]:
imd_df = pd.read_csv("/Users/mahnoorshahzad/Documents/a3_year/ASDS/IMD_ward.csv")

In [ ]:
newestcrime = pd.read_csv("/Users/mahnoorshahzad/Documents/a3_year/newestcrime.csv")

In [ ]:
lsoa_df = pd.read_csv("/Users/mahnoorshahzad/Documents/a3_year/ASDS/LSOA_Data.csv")

In [ ]:
newpopden = pd.read_csv("/Users/mahnoorshahzad/Documents/a3_year/newpopden.csv")

In [ ]:
newestcrime = newestcrime.rename(columns={'avg_2020.1': 'avg_2021'})

In [ ]:
lsoa_df.columns

In [ ]:
newpopden.head()

In [ ]:
newpopden.columns

In [ ]:
newpopden = newpopden.drop(columns= ['Unnamed: 5',
       'Unnamed: 6'])

In [ ]:
lsoa_df = lsoa_df.drop(columns= [
       'RGN11CD', 'RGN11NM', 'USUALRES', 'HHOLDRES', 'COMESTRES', 'POPDEN',
       'HHOLDS', 
       'Owned', 'SharedOwnshp', 'SocialRented', 'PrivateRented', 'RentFree',
       'ORroomsM1', 'ORbedsM1', 'RoomsHH', 'BedsHH', 'White', 'MultiEthnic',
       'Asian', 'Black', 'OtherEthnic', 'MedianIncome', 'SumPrice',
       'MeanPrice', 'MedianPrice', 'EntireHome', 'PrivateRoom', 'SharedRoom',
       'SmallHost', 'MultiHost', 'PropertyCount', 'Area'])

In [ ]:
new_lsoa = pd.merge(lsoa_df, newpopden, on='LSOA11CD', left_index=False)

In [ ]:
new_lsoa.head()

In [ ]:
crimelsoa_map = pd.merge(new_lsoa, newestcrime, on='LSOA11CD', left_index=False)

In [ ]:
crime_imd_lsoa = pd.merge(crimelsoa_map, imd_df, on='LSOA11CD', left_index=False)

In [ ]:
crime_imd_lsoa['geometry'] = crime_imd_lsoa['geometry'].apply(lambda x: loads(x)) 
gdf = crime_imd_lsoa.set_geometry('geometry')

In [ ]:
crimelsoa_map['geometry'] = crimelsoa_map['geometry'].apply(lambda x: loads(x)) 
gdf2 = crimelsoa_map.set_geometry('geometry')

In [ ]:
gdf2['avg2019_den'] = gdf2['avg_2019'] / gdf2['Area']
gdf2['avg2020_den'] = gdf2['avg_2020'] / gdf2['Area']
gdf2['avg2021_den'] = gdf2['avg_2021'] / gdf2['Area']
gdf2['avg2019_prop'] = (gdf2['avg_2019'] / gdf2['USUALRES'])*100
gdf2['avg2020_prop'] = (gdf2['avg_2020'] / gdf2['USUALRES'])*100
gdf2['avg2021_prop'] = (gdf2['avg_2021'] / gdf2['USUALRES'])*100

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(12,10))
gdf2.plot(ax=ax1, 
         column= 'avg2019_prop', scheme= 'fisher_jenks', 
        cmap = pcs.Reds_7.mpl_colormap,
        edgecolor='grey', linewidth=0.1,
        legend=True,legend_kwds={'title': " Crime Proportion(%)",'loc': 'lower right'})
plt.suptitle('Violent Crime Percentage in London 2019', size=20)
plt.axis('off')
ax1.annotate('Source: London Datastore (2022)',
             xy=(0.1, 0.1), xycoords='figure fraction', 
             horizontalalignment='left', verticalalignment='top', 
             fontsize=12, color='#555555')
plt.savefig("/Users/mahnoorshahzad/Documents/a3_year/ASDS/prop2019", dpi= 300, bbox_inches= 'tight', facecolour="white" )
plt.show()

fig, ax1 = plt.subplots(1, figsize=(12,10))
gdf2.plot(ax=ax1, 
         column= 'avg2020_prop', scheme= 'fisher_jenks',
        cmap = pcs.Reds_7.mpl_colormap,
        edgecolor='grey', linewidth=0.1,
        legend=True, legend_kwds={'title': " Crime Proportion(%)",'loc': 'lower right'})
plt.suptitle('Violent Crime Percentage in London 2020', size=20)
plt.axis('off')
ax1.annotate('Source: London Datastore (2022)',
             xy=(0.1, 0.1), xycoords='figure fraction', 
             horizontalalignment='left', verticalalignment='top', 
             fontsize=12, color='#555555')

plt.savefig("/Users/mahnoorshahzad/Documents/a3_year/ASDS/prop2020", dpi= 300, bbox_inches= 'tight', facecolour="white" )
plt.show()

fig, ax1 = plt.subplots(1, figsize=(12,10))
gdf2.plot(ax=ax1, 
         column= 'avg2021_prop', scheme= 'fisher_jenks',
        cmap = pcs.Reds_7.mpl_colormap,
        edgecolor='grey', linewidth=0.1,
        legend=True, legend_kwds={'title': " Crime Proportion(%)",'loc': 'lower right'})
plt.suptitle('Violent Crime Percentage in London 2021', size=20)
plt.axis('off')
ax1.annotate('Source: London Datastore (2022)',
             xy=(0.1, 0.1), xycoords='figure fraction', 
             horizontalalignment='left', verticalalignment='top', 
             fontsize=12, color='#555555')
plt.savefig("/Users/mahnoorshahzad/Documents/a3_year/ASDS/prop2021", dpi= 300, bbox_inches= 'tight', facecolour="white" )
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(12,10))
gdf2.plot(ax=ax1, 
         column= 'avg2019_den', scheme= 'fisher_jenks', 
        cmap = pcs.Reds_7.mpl_colormap,
        edgecolor='grey', linewidth=0.1,
        legend=True)
plt.suptitle('Violent Crime Density in London 2019', size=20)
plt.axis('off')
ax1.annotate('Source: London Datastore (2022)',
             xy=(0.1, 0.1), xycoords='figure fraction', 
             horizontalalignment='left', verticalalignment='top', 
             fontsize=12, color='#555555')
plt.savefig("/Users/mahnoorshahzad/Documents/a3_year/ASDS/den2019", dpi= 300, bbox_inches= 'tight', facecolour="white" )
plt.show()

fig, ax1 = plt.subplots(1, figsize=(12,10))
gdf2.plot(ax=ax1, 
         column= 'avg2020_den', scheme= 'fisher_jenks',
        cmap = pcs.Reds_7.mpl_colormap,
        edgecolor='grey', linewidth=0.1,
        legend=True)
plt.suptitle('Violent Crime Density in London 2020', size=20)
plt.axis('off')
ax1.annotate('Source: London Datastore (2022)',
             xy=(0.1, 0.1), xycoords='figure fraction', 
             horizontalalignment='left', verticalalignment='top', 
             fontsize=12, color='#555555')
plt.savefig("/Users/mahnoorshahzad/Documents/a3_year/ASDS/den2020", dpi= 300, bbox_inches= 'tight', facecolour="white" )
plt.show()

fig, ax1 = plt.subplots(1, figsize=(12,10))
gdf2.plot(ax=ax1, 
         column= 'avg2021_den', scheme= 'fisher_jenks',
        cmap = pcs.Reds_7.mpl_colormap,
        edgecolor='grey', linewidth=0.1,
        legend=True)
plt.suptitle('Violent Crime Density in London 2021', size=20)
plt.axis('off')
ax1.annotate('Source: London Datastore (2022)',
             xy=(0.1, 0.1), xycoords='figure fraction', 
             horizontalalignment='left', verticalalignment='top', 
             fontsize=12, color='#555555')
plt.savefig("/Users/mahnoorshahzad/Documents/a3_year/ASDS/den2021", dpi= 300, bbox_inches= 'tight', facecolour="white" )
plt.show()

In [ ]:
import esda
import pysal

In [ ]:
w = weights.KNN.from_dataframe(gdf, k=8) 
w.transform = 'R'
moran = esda.moran.Moran(gdf['avg_2021'], w)

In [ ]:
gdf['avg_2021'].median()

In [ ]:
binary = gdf['avg_2021']> gdf['avg_2021'].median()
sum(binary)

In [ ]:
labels = ['Low Price', 'High Price']
binary = [labels[i] for i in 1*binary] 
gdf['binary'] = binary

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = plt.gca()
gdf.plot(column='binary', cmap='binary', edgecolor='grey', legend=True, ax=ax)

In [ ]:
w_KNN1 = lps.weights.KNN.from_dataframe(gdf)

In [ ]:
from pysal.explore import esda 
binary = 1 * (gdf2['avg_2021']> gdf2['avg_2021'].median()) # convert back to binary
w_KNN = lps.weights.KNN.from_dataframe(gdf2)
w_KNN.transform = 'R'
np.random.seed(12345)
jc = esda.join_counts.Join_Counts(binary, w_KNN)

In [ ]:
jc.bb

In [ ]:
jc.ww

In [ ]:
jc.bw

In [ ]:
jc.bb + jc.ww + jc.bw

In [ ]:
jc.p_sim_bb

In [ ]:
w_KNN.s0 / 2

In [ ]:
mi = esda.moran.Moran(gdf2['avg_2021'], w_KNN) # call moran function
mi.I # print out the moran's I value

In [ ]:
sns.kdeplot(mi.sim, shade=True)
plt.vlines(mi.I, 0, 40, color='r')
plt.vlines(mi.EI, 0, 40)
plt.xlabel("Moran's I")

In [ ]:
mi.p_sim

In [ ]:
lisa = ps.explore.esda.Moran_Local(gdf2['avg_2021'].values, w_KNN, permutations=999)
lisa.Is

In [ ]:
lisa.q

In [ ]:
lisa.p_sim

In [ ]:
(lisa.p_sim < 0.05).sum()

In [ ]:
from pysal import lib

In [ ]:
gdf2['lag_avg'] = ps.lib.weights.lag_spatial(w_KNN, gdf2['avg_2021'])
sigs = gdf2['avg_2021'][lisa.p_sim <= .05]
W_sigs = gdf2['lag_avg'][lisa.p_sim <= .05]
insigs = gdf2['avg_2021'][lisa.p_sim > .05]
W_insigs = gdf2['lag_avg'][lisa.p_sim > .05]

In [ ]:
gdf2['lag_avg'] = ps.lib.weights.lag_spatial(w_KNN, gdf2['avg_2020'])
sigs = gdf2['avg_2020'][lisa.p_sim <= .05]
W_sigs = gdf2['lag_avg'][lisa.p_sim <= .05]
insigs = gdf2['avg_2020'][lisa.p_sim > .05]
W_insigs = gdf2['lag_avg'][lisa.p_sim > .05]

In [ ]:
from splot.esda import lisa_cluster
fig = lisa_cluster(lisa, gdf2)
plt.title("LISA Cluster Map")
plt.show()

In [ ]:
sig = lisa.p_sim < 0.05
hotspot = sig * lisa.q==1
coldspot = sig * lisa.q==3
doughnut = sig * lisa.q==2
diamond = sig * lisa.q==4

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 8))
# Plot building blocks
gdf2.plot(ax=ax, facecolor='1', linewidth=0.1)
# Plot HH clusters
hh = gdf2.loc[(lisa.q==1) & (sig==True), 'geometry']
hh.plot(ax=ax, color='red', linewidth=0.1, edgecolor='w')
# Plot LL clusters
ll = gdf2.loc[(lisa.q==3) & (sig==True), 'geometry']
ll.plot(ax=ax, color='blue', linewidth=0.1, edgecolor='w')
# Plot LH clusters
lh = gdf2.loc[(lisa.q==2) & (sig==True), 'geometry']
lh.plot(ax=ax, color='#83cef4', linewidth=0.1, edgecolor='w')
# Plot HL clusters
hl = gdf2.loc[(lisa.q==4) & (sig==True), 'geometry']
hl.plot(ax=ax, color='pink', linewidth=0.1, edgecolor='w')
# Non-significant
ns = gdf2.loc[sig!=True, 'geometry']
ns.plot(ax=ax, color='0.75', linewidth=0.1, edgecolor='w')
# Style and draw
f.suptitle('LISA for Average Crime in London 2021', size=20)
f.set_facecolor('w')
plt.axis('off')
f.savefig("/Users/mahnoorshahzad/Documents/a3_year/ASDS/lisa2021", dpi= 300, bbox_inches= 'tight', facecolour="white" )
plt.show()

In [ ]:
gdf.hist('avg_2021')

In [ ]:
gdf.hist('imd_score')

In [ ]:
gdf.hist('POPDEN')

In [ ]:
gdf.hist('RoadsArea')

In [ ]:
gdf.hist('AVHHOLDSZ')

In [ ]:
gdf.hist('emp_score')

In [ ]:
gdf.hist('edu_score')

In [ ]:
gdf['sqGreenspaceArea'] = np.sqrt(gdf['GreenspaceArea'])
gdf.hist('GreenspaceArea')

In [ ]:
gdf['logavg_2021'] = np.log(gdf['avg_2021'])
gdf['sqPOPDEN'] = np.sqrt(gdf['POPDEN'])
gdf['sqRoadsArea'] = np.sqrt(gdf['RoadsArea'])
gdf['sqincome_score'] = np.sqrt(gdf['income_score'])
gdf['sqedu_score'] = np.sqrt(gdf['edu_score'])

In [ ]:
gdf.shape

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
 
y, X = dmatrices('logavg_2021 ~ sqedu_score  + sqRoadsArea + housing_score + sqPOPDEN + heathndis_score + living_score + emp_score + living_score + sqincome_score', data=gdf, return_type='dataframe')


vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

 
vif

In [ ]:
crime = gdf['logavg_2021']
y = np.array(crime)
y.shape = (len(crime),1)

X= []

X.append(gdf['sqedu_score'])
X.append(gdf['sqRoadsArea']) 
X.append(gdf['sqPOPDEN']) 
X.append(gdf['housing_score']) 
X.append(gdf['emp_score'])
X.append(gdf['heathndis_score'])
X.append(gdf['living_score'])

X = np.array(X).T

In [ ]:
m1 = OLS(y, X, name_y='logavg_2021',
         name_x=['sqedu_score','sqRoadsArea', 'sqPOPDEN', 'housing_score', 'emp_score', 'heathndis_score', 'living_score'], 
         name_ds='gdf')
print(m1.summary)

In [ ]:
spat_err = ML_Error(y,X,w_KNN1, name_y='logavg_2021', 
                    name_x=['sqedu_score','sqRoadsArea', 'sqPOPDEN', 'housing_score', 'emp_score', 'heathndis_score', 'living_score'], 
                    name_w='w_KNN1', name_ds='gdf')
print(spat_err.summary)

In [ ]:
spat_lag = ML_Lag(y,X,w_KNN1,name_y='logavg_2021', 
                  name_x=['sqedu_score','sqRoadsArea', 'sqPOPDEN', 'housing_score', 'emp_score', 'heathndis_score', 'living_score' ],
                  name_w='w_KNN1',
                  name_ds='gdf')
print(spat_lag.summary)

In [ ]:
m1 = OLS(y, X, name_y='logavg_2021',
         name_x=['sqedu_score','sqRoadsArea', 'sqPOPDEN', 'housing_score', 'emp_score', 'heathndis_score', 'living_score'], 
         name_ds='gdf')
sl = ML_Lag(y,X,w_KNN1,name_y='logavg_2021', 
                  name_x=['sqedu_score','sqRoadsArea', 'sqPOPDEN', 'housing_score', 'emp_score', 'heathndis_score', 'living_score' ],
                  name_w='w_KNN1',
                  name_ds='gdf')
se = ML_Error(y,X,w_KNN1, name_y='logavg_2021', 
                    name_x=['sqedu_score','sqRoadsArea', 'sqPOPDEN', 'housing_score', 'emp_score', 'heathndis_score', 'living_score'], 
                    name_w='w_KNN1', name_ds='gdf')

In [ ]:
from sklearn.metrics import mean_squared_error as mse

mses = pd.Series({'OLS': mse(y, m1.predy.flatten()), 
                  'SL': mse(y, sl.predy.flatten()), 
                  'SE': mse(y, se.predy.flatten())
                    })
mses.sort_values()